# Autonomous Underwater Vehicle Fault Diagnosis

Ji, Daxiong (2021), “Autonomous Underwater Vehicle Fault Diagnosis Dataset”, Mendeley Data, V1, doi: 10.17632/7rp2pmr6mx.1

Dataset source: https://data.mendeley.com/datasets/7rp2pmr6mx/1

In [70]:
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sktime.datatypes import convert_to
from sktime.datatypes._panel._convert import (
    from_3d_numpy_to_nested,
    from_3d_numpy_to_multi_index,
    from_multi_index_to_nested
)

%matplotlib inline

## Training dataset

In [20]:
!ls -l /mnt/f/Downloads/Autonomous-Underwater-Vehicle-Fault-Diagnosis-Dataset/Dataset/train/

total 0
drwxrwxrwx 1 us3r us3r 4096 Feb 27  2021 AddWeight
drwxrwxrwx 1 us3r us3r 4096 Feb 27  2021 Normal
drwxrwxrwx 1 us3r us3r 4096 Feb 27  2021 PressureGain_constant
drwxrwxrwx 1 us3r us3r 4096 Feb 27  2021 PropellerDamage_bad
drwxrwxrwx 1 us3r us3r 4096 Feb 27  2021 PropellerDamage_slight


In [58]:
%%time

def get_label(file_path):
    return file_path.split("/")[-2]


def reorder_columns(columns):
    cols = list(columns)
    return cols[-2:] + cols[:-2]


df_train = pd.DataFrame()
labels_train = []
instance_id = 0
DF_LENGTH = 180

dataset_path = '/mnt/f/Downloads/Autonomous-Underwater-Vehicle-Fault-Diagnosis-Dataset/Dataset/train/**/*.csv'

for file_path in glob.glob(dataset_path, recursive=True):
    df_tmp = pd.read_csv(file_path)
    
    if len(df_tmp) < DF_LENGTH:
        continue
    
    df_tmp = df_tmp[0:DF_LENGTH]
    df_tmp["instances"] = instance_id
    df_tmp["timepoints"] = list(range(0, DF_LENGTH))
    
    cols = reorder_columns(df_tmp.columns)
    df_tmp = df_tmp[cols]
    
    df_train = pd.concat([df_train, df_tmp])
    df_train = df_train.reset_index(drop=True)
    labels_train.append(get_label(file_path))
    instance_id += 1


df_train.shape, len(labels_train)

CPU times: user 46.9 s, sys: 8.22 s, total: 55.2 s
Wall time: 1min 11s


((175500, 19), 975)

In [72]:
df_train = df_train.set_index(["instances", "timepoints"])
df_train

time  pwm1  pwm2  pwm3  pwm4  depth  press  voltage  \
instances timepoints                                                          
0         0           202409  1100  1100  1100  1100  -0.02    592    12.08   
          1           202409  1100  1100  1100  1100  -0.01    593    12.08   
          2           202409  1100  1100  1100  1100   0.00    594    12.06   
          3           202409  1183  1183  1183  1183   0.01    595    12.05   
          4           202409  1183  1183  1183  1183   0.01    595    11.97   
...                      ...   ...   ...   ...   ...    ...    ...      ...   
974       175         205536  1200  1200  1200  1200   0.21    632    11.86   
          176         205536  1200  1200  1200  1200   0.21    632    11.86   
          177         205536  1200  1200  1200  1200   0.20    631    11.86   
          178         205536  1200  1200  1200  1200   0.19    630    11.85   
          179         205536  1200  1200  1200  1200   0.19    630    11.86   

                      roll  pitch    yaw    a_x   a_y    a_z  w_row  w_pitch  \
instances timepoints                                                           
0         0           5.97  -0.06  37.16  -0.02  0.90   8.76    1.6     -2.1   
          1           6.05  -0.19  37.16   0.13  0.95   8.56    1.5     -3.0   
          2           6.12  -0.34  37.17   0.07  0.88   8.64    1.3     -3.4   
          3           6.18  -0.51  37.17   0.09  0.96   8.80    0.6     -3.3   
          4           6.21  -0.66  37.17   0.07  1.00   8.61    0.2     -2.7   
...                    ...    ...    ...    ...   ...    ...    ...      ...   
974       175         5.63  -1.66 -80.80   6.89  0.67   5.65   -0.2     -0.4   
          176         5.45  -2.15 -80.46  13.20 -1.85   5.97   -1.3     -2.6   
          177         5.31  -2.95 -80.19  -8.52  1.55  11.26   -2.4    -10.8   
          178         5.16  -3.39 -80.00   9.64  1.47   6.45   -2.6     -4.3   
          179         5.06  -3.73 -79.92   9.35 -0.59   6.94   -3.2     -1.5   

                      w_yaw  
instances timepoints         
0         0             0.3  
          1             0.4  
          2             0.2  
          3             0.3  
          4             0.1  
...                     ...  
974       175           8.0  
          176           5.9  
          177           6.3  
          178           4.3  
          179           1.1  

[175500 rows x 17 columns]

In [76]:
preprocessed_path = "/mnt/f/Downloads/Autonomous-Underwater-Vehicle-Fault-Diagnosis-Dataset/Dataset/preprocessed/"
df_train.to_csv(preprocessed_path + "features-train.csv", index=True)

In [80]:
pd.DataFrame(labels_train, columns=["label"]).to_csv(preprocessed_path + "labels-train.csv", index=False)

## Test datasets

In [25]:
!ls -l /mnt/f/Downloads/Autonomous-Underwater-Vehicle-Fault-Diagnosis-Dataset/Dataset/test/

total 0
drwxrwxrwx 1 us3r us3r 4096 Feb 27  2021 AddWeight
drwxrwxrwx 1 us3r us3r 4096 Feb 27  2021 Normal
drwxrwxrwx 1 us3r us3r 4096 Feb 27  2021 PressureGain_constant
drwxrwxrwx 1 us3r us3r 4096 Feb 27  2021 PropellerDamage_bad
drwxrwxrwx 1 us3r us3r 4096 Feb 27  2021 PropellerDamage_slight


In [81]:
%%time

def get_label(file_path):
    return file_path.split("/")[-2]


def reorder_columns(columns):
    cols = list(columns)
    return cols[-2:] + cols[:-2]


df_test = pd.DataFrame()
labels_test = []
instance_id = 0
DF_LENGTH = 180

dataset_path = '/mnt/f/Downloads/Autonomous-Underwater-Vehicle-Fault-Diagnosis-Dataset/Dataset/test/**/*.csv'

for file_path in glob.glob(dataset_path, recursive=True):
    df_tmp = pd.read_csv(file_path)
    
    if len(df_tmp) < DF_LENGTH:
        continue
    
    df_tmp = df_tmp[0:DF_LENGTH]
    df_tmp["instances"] = instance_id
    df_tmp["timepoints"] = list(range(0, DF_LENGTH))
    
    cols = reorder_columns(df_tmp.columns)
    df_tmp = df_tmp[cols]
    
    df_test = pd.concat([df_test, df_tmp])
    df_test = df_test.reset_index(drop=True)
    labels_test.append(get_label(file_path))
    instance_id += 1


df_test.shape, len(labels_test)

CPU times: user 4.6 s, sys: 449 ms, total: 5.05 s
Wall time: 8.49 s


((44100, 19), 245)

In [82]:
df_test = df_test.set_index(["instances", "timepoints"])
df_test

time  pwm1  pwm2  pwm3  pwm4  depth  press  voltage  \
instances timepoints                                                          
0         0           213241  1100  1100  1100  1100   0.00    601    12.08   
          1           213241  1100  1100  1100  1100   0.00    601    12.07   
          2           213241  1100  1100  1100  1100   0.01    602    12.07   
          3           213241  1100  1100  1100  1100   0.00    601    12.04   
          4           213242  1183  1183  1183  1183   0.04    605    11.99   
...                      ...   ...   ...   ...   ...    ...    ...      ...   
244       175         203712  1200  1200  1200  1200   0.40    652    11.92   
          176         203712  1200  1200  1200  1200   0.42    655    11.93   
          177         203712  1200  1200  1200  1200   0.43    656    11.92   
          178         203712  1200  1200  1200  1200   0.37    649    11.91   
          179         203712  1200  1200  1200  1200   0.37    649    11.93   

                      roll  pitch    yaw    a_x   a_y    a_z  w_row  w_pitch  \
instances timepoints                                                           
0         0           6.24  -3.20 -64.47   0.52  1.03   8.88    0.4     -1.5   
          1           6.27  -3.27 -64.48   0.59  0.97   8.80    0.5     -1.2   
          2           6.30  -3.30 -64.48   0.31  0.81   8.66    0.1     -1.4   
          3           6.38  -3.30 -64.43  -0.49  0.05   8.59    3.1      2.4   
          4           6.64  -3.04 -64.27  13.69  0.68   3.55    7.4      8.4   
...                    ...    ...    ...    ...   ...    ...    ...      ...   
244       175         2.72   1.45  64.90   2.20 -4.80  16.57    6.2     -6.9   
          176         2.89   2.21  65.51  -0.42 -1.55   7.20    3.8      8.4   
          177         3.14   1.45  66.15  -2.14  2.00  10.96    3.2     -2.1   
          178         3.66   1.66  66.95   2.14  0.41   9.49    1.8     -0.6   
          179         3.78   0.95  67.74  -2.90  2.78   8.65    3.6      1.7   

                      w_yaw  
instances timepoints         
0         0            -0.1  
          1             0.1  
          2             0.1  
          3             2.0  
          4             2.3  
...                     ...  
244       175          11.7  
          176          12.3  
          177          16.0  
          178          17.5  
          179          16.6  

[44100 rows x 17 columns]

In [83]:
preprocessed_path = "/mnt/f/Downloads/Autonomous-Underwater-Vehicle-Fault-Diagnosis-Dataset/Dataset/preprocessed/"
df_test.to_csv(preprocessed_path + "features-test.csv", index=True)

In [84]:
pd.DataFrame(labels_test, columns=["label"]).to_csv(preprocessed_path + "labels-test.csv", index=False)